In [3]:
import pandas as pd
import janitor as jn
import numpy as np
from janitor.functions._numba import _numba_less_than, _numba_greater_than

In [4]:
df1 = pd.DataFrame({"id": [1, 1, 1, 2, 2, 3], "value_1": [2, 5, 7, 1, 23, 4]})
df2 = pd.DataFrame(
    {
        "id": [1, 1, 1, 1, 2, 2, 2, 3],
        "value_2A": [0, 3, 7, 12, 0, 2, 3, 1],
        "value_2B": [1, 5, 9, 15, 1, 4, 6, 3],
    }
)

In [5]:
df1

,id,value_1
0,1,2
1,1,5
2,1,7
3,2,1
4,2,23
5,3,4


In [6]:
df2

,id,value_2A,value_2B
0,1,0,1
1,1,3,5
2,1,7,9
3,1,12,15
4,2,0,1
5,2,2,4
6,2,3,6
7,3,1,3


In [19]:
df2 = df2.sort_values('value_2A')

In [20]:
df2

,id,value_2A,value_2B
0,1,0,1
4,2,0,1
7,3,1,3
5,2,2,4
1,1,3,5
6,2,3,6
2,1,7,9
3,1,12,15


In [9]:
df1.conditional_join(df2, ('value_1', 'value_2A','>'), ('value_1','value_2B', '<'), use_numba=True,keep='all')

left         right                  
    id value_1    id value_2A value_2B
0    1       2     3        1        3
1    1       5     2        3        6
2    3       4     1        3        5
3    3       4     2        3        6

In [10]:
df1.conditional_join(df2, ('value_1', 'value_2A','>'), use_numba=False,keep='all')

left         right                  
     id value_1    id value_2A value_2B
0     1       2     1        0        1
1     1       2     2        0        1
2     1       2     3        1        3
3     1       5     1        0        1
4     1       5     2        0        1
5     1       5     3        1        3
6     1       5     2        2        4
7     1       5     1        3        5
8     1       5     2        3        6
9     1       7     1        0        1
10    1       7     2        0        1
11    1       7     3        1        3
12    1       7     2        2        4
13    1       7     1        3        5
14    1       7     2        3        6
15    2       1     1        0        1
16    2       1     2        0        1
17    2      23     1        0        1
18    2      23     2        0        1
19    2      23     3        1        3
20    2      23     2        2        4
21    2      23     1        3        5
22    2      23     2        3        6
23    2      23     1        7        9
24    2      23     1       12       15
25    3       4     1        0        1
26    3       4     2        0        1
27    3       4     3        1        3
28    3       4     2        2        4
29    3       4     1        3        5
30    3       4     2        3        6

In [11]:
df1.value_1.array

<NumpyExtensionArray>
[2, 5, 7, 1, 23, 4]
Length: 6, dtype: int64

In [12]:
np.sort(df2['value_2A'])

array([ 0,  0,  1,  2,  3,  3,  7, 12])

In [13]:
_numba_greater_than(np.sort(df2['value_2B']), 6)

6

In [14]:
df = {
    "A": [-1,0],
    # "B": [1.0],
    # "C": [""],
    # "D": [False],
    # "E": [pd.to_datetime("1970-01-01")],
}
df = pd.DataFrame(df)
df

,A
0,-1
1,0


In [15]:
right = {
    "Integers": [0,0],
    "Numeric": [1,0.0],
    "Floats": [0.0,0.0],
    "Strings": ["",""],
    "Booleans": [False,False],
    "Date": [pd.to_datetime("1970-01-01")]*2,
    "Dates_Right": [pd.to_datetime("1970-01-01")]*2,
}
right = pd.DataFrame(right)
right

,Integers,Numeric,Floats,Strings,Booleans,Date,Dates_Right
0,0,1.0,0.0,,False,1970-01-01,1970-01-01
1,0,0.0,0.0,,False,1970-01-01,1970-01-01


In [16]:
# df = df.sort_values("B").dropna(subset=["B"])
actual = (
        df[["A"]]
        .conditional_join(
            right[["Integers"]],
            ("A", "Integers", "<"),
            how="inner",
            # keep="first",
            use_numba=True,
        )
        # .sort_values(["A", "Integers"], ignore_index=True)
    )

actual

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


,A,Integers
0,-1,0
1,-1,0


In [17]:
(
        df[["A"]]
        .conditional_join(
            right[["Integers"]],
            ("A", "Integers", "<"),
            how="inner",
            # keep="first",
            use_numba=False,
        )
     #   .sort_values(["B", "Numeric"], ignore_index=True)
    )

,A,Integers
0,-1,0
1,-1,0


In [18]:
(
        df[["A"]]
        .merge(right[["Integers"]], how="cross")
        .loc[lambda df: df.A.lt(df.Integers)]
        .sort_values(["A", "Integers"], ignore_index=True)
    )

,A,Integers
0,-1,0
1,-1,0
